## Time Series Project

1. Explore the Electricity Dataset, using tools like line charts and decomposition
2. Forecast the first three days of February (72 hours) of Electricity consumption for column `PowerConsumption_Zone1`, using the entire month of January to train your Model.
3. Linear Regression and Facebook Prophet. Compare their accuracy using plots of predicted vs. actual, as well as metrics like MAE and MAPE


In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.api as sm

electricity_df = pd.read_csv(
    "../Data/powerconsumption.csv", 
    index_col=["Datetime"],
    parse_dates=["Datetime"]
).resample("H").mean()

electricity_df.head()

## EDA

In [0]:
electricity_df["PowerConsumption_Zone1"].iloc[:744].plot()

In [0]:
from statsmodels.tsa.seasonal import seasonal_decompose

results = seasonal_decompose(electricity_df["PowerConsumption_Zone1"].iloc[:744], period=(24 * 7))

results.plot();

## Linear Regression

In [0]:
electricity_df = electricity_df["PowerConsumption_Zone1"].reset_index()

electricity_df = electricity_df.assign(
    trend = electricity_df.index,
    hour = electricity_df["Datetime"].dt.hour.astype("string"),
#     day_of_week = electricity_df["Datetime"].dt.dayofweek.astype("string"),
).set_index("Datetime")

electricity_df = pd.get_dummies(electricity_df, drop_first=True)

electricity_df.head()

electricity_df.info()

In [0]:
electricity_train = electricity_df.loc[:"2017-01-31"]
electricity_test = electricity_df.loc["2017-02-01": "2017-02-03"]

X_train = sm.add_constant(electricity_train.drop("PowerConsumption_Zone1", axis=1))
y_train = electricity_train["PowerConsumption_Zone1"]

X_test = sm.add_constant(electricity_test.drop("PowerConsumption_Zone1", axis=1))
y_test = electricity_test["PowerConsumption_Zone1"]

In [0]:
import statsmodels.api as sm

model = sm.OLS(y_train, X_train).fit()
model.summary()

In [0]:
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_absolute_percentage_error as mape

print(f"MAE: {mae(y_test, model.predict(X_test))}")
print(f"MAPE: {mape(y_test, model.predict(X_test))}")

In [0]:
test_preds = pd.DataFrame({
    "actuals": y_test.values, 
    "predicted": model.predict(X_test)
})

test_preds.plot(ylim=0)

## Facebook Prophet

In [0]:
from prophet import Prophet

electricity_prophet = (
    electricity_df
    .reset_index()
    .rename({"Datetime": "ds", "PowerConsumption_Zone1": "y"}, axis=1)
)

electricity_train = electricity_prophet.query("ds < '2017-02-01'")   
electricity_test = electricity_prophet.query("'2017-02-01' <= ds < '2017-02-04'")
                                            
electricity_test.tail()                                              

In [0]:
model = Prophet()
model.fit(electricity_train[["ds", "y"]])

In [0]:
future = model.make_future_dataframe(periods=72, freq="H")

forecast = model.predict(future)

model.plot(forecast);

In [0]:
model.plot_components(forecast);

In [0]:
electricity_score = (
electricity_test
 .assign(predictions = model.predict(future)["yhat"])
 .set_index("ds")
 .loc[:,["y", "predictions"]]
)

electricity_score.plot()

sns.despine()

In [0]:
print(f"MAE: {mae(electricity_score['y'], electricity_score['predictions'])}")
print(f"MAPE: {mape(electricity_score['y'], electricity_score['predictions'])}")